Basic notebook to test the C bindings of Julia language prior to launching the parsing of the language into IL, which is expected by tapenade.

Note that it is not trivial to get to pass a function name (which should be a symbol) as a String. More on this later. 

In [1]:
#= 
c_libname = "./libcnt"
c_incr = "calling_c"
c_refreturn = "returnbyref"
c_return_val = "returncnt"
c_big_update = "big_update"
c_print = "print_in_c"

x=ccall((Meta.parse(c_big_update),c_libname),Int32,(Int32,), 5)
x= ccall((c_return_val,c_libname),UInt32,())
x=ccall((:big_update, "./libcnt"),Int32,(Int32,), 5)
x=ccall((:calling_c, "./libcnt"),Int32,())
ccall((:print_in_c, "./libcnt"),Int32,())
=#

One way to get around it is to pass a function handler...

In [2]:
#= For first timers
using Pkg
Pkg.add("Libdl")
=#

In [3]:
using Libdl

In [4]:
c_libname = "./libcnt"
lib = Libdl.dlopen("./libcnt.so") # Open the library explicitly.

Ptr{Nothing} @0x00000000018a21d0

In [5]:
# List all my functions
c_big_update = Libdl.dlsym(lib, :big_update)   # Get a symbol for the function to call.
c_incr = Libdl.dlsym(lib, :calling_c)   # Get a symbol for the function to call.
c_refreturn = Libdl.dlsym(lib, :returnbyref)   # Get a symbol for the function to call.
c_print = Libdl.dlsym(lib, :print_in_c)
c_valreturn = Libdl.dlsym(lib, :returncnt)

Ptr{Nothing} @0x00007efe94e0429c

In [6]:
x = ccall(c_print, Int32, ())

		***** cnt is 0


0

In [7]:
ccall(c_incr, Int32, ())

1

In [8]:
x = ccall(c_print, Int32, ())

		***** cnt is 1


0

In [9]:
isOk = ccall(c_big_update,Int32,(Int32,), 5)

0

In [10]:
x = ccall(c_print, Int32, ())

		***** cnt is 6


0

In [11]:
cur_cnt = ccall(c_valreturn, Int32, ())

6

In [12]:
v = Array{Int32,1}([1])

1-element Vector{Int32}:
 1

In [13]:
return_val = ccall(c_refreturn, Int32, (Ptr{Cint},), v)

0

In [14]:
v

1-element Vector{Int32}:
 6

In [15]:
cur_cnt = ccall(c_valreturn, Int32, ())

6

Let's try playing with more complex objects passed as parameters / return values 

In [16]:
struct BinaryTree
    v::Cfloat
    lchild::Ptr{BinaryTree}
    rchild::Ptr{BinaryTree}
end

In [23]:
# List all my functions
c_new_tree = Libdl.dlsym(lib, :new_tree)
c_add_left = Libdl.dlsym(lib, :add_left)
c_add_right = Libdl.dlsym(lib, :add_right)
c_print_to_the_left = Libdl.dlsym(lib, :print_to_the_left)
c_print_to_the_right = Libdl.dlsym(lib, :print_to_the_right)

Ptr{Nothing} @0x00007efe94e041c4

In [24]:
# Start with handling a binary tree in Julia, from a ressource created in C: 
from_c_bintree = ccall(c_new_tree, Ptr{BinaryTree}, (Cfloat,), 3.0)

Ptr{BinaryTree} @0x0000000001ce16a0

In [25]:
ccall(c_print_to_the_left, Cvoid, (Ref{BinaryTree},), from_c_bintree)

Value number 0 is 3.000000


In [26]:
ccall(c_print_to_the_right, Cvoid, (Ref{BinaryTree},), from_c_bintree)

Value number 0 is 3.000000


In [27]:
# Create a right child straigth from Julia, and add it to the C based tree
julia_right_child = BinaryTree(42.0, Ptr{BinaryTree}(C_NULL), Ptr{BinaryTree}(C_NULL))

BinaryTree(42.0f0, Ptr{BinaryTree} @0x0000000000000000, Ptr{BinaryTree} @0x0000000000000000)

In [28]:
# Create a right child straigth from Julia, and add it to the C based tree
c_left_child = ccall(c_new_tree, Ptr{BinaryTree}, (Cfloat,), 13.43)

Ptr{BinaryTree} @0x00000000016c4150

In [29]:
return_val = ccall(c_add_left, Int32, (Ref{BinaryTree},Ref{BinaryTree},), from_c_bintree, c_left_child)

0

In [30]:
println("Calling to the left")
ccall(c_print_to_the_left, Cvoid, (Ref{BinaryTree},), from_c_bintree)
println("Calling to the right")
ccall(c_print_to_the_right, Cvoid, (Ref{BinaryTree},), from_c_bintree)

Calling to the left
Calling to the right
Value number 0 is 3.000000
Value number 1 is 13.430000
Value number 0 is 3.000000


In [31]:
return_val = ccall(c_add_right, Int32, (Ref{BinaryTree},Ref{BinaryTree},), from_c_bintree, julia_right_child)

0

In [32]:
println("Calling to the left")
ccall(c_print_to_the_left, Cvoid, (Ref{BinaryTree},), from_c_bintree)
println("Calling to the right")
ccall(c_print_to_the_right, Cvoid, (Ref{BinaryTree},), from_c_bintree)

Calling to the left
Calling to the right
Value number 0 is 3.000000
Value number 1 is 13.430000
Value number 0 is 3.000000
Value number 1 is 42.000000


In [33]:
Libdl.dlclose(lib) # Close the library explicitly.

true